In [2]:
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageEnhance
import face_recognition
import numpy as np


class FaceDetectionTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Detection Enhancement Tool")
        self.image_path = None
        self.original_image = None
        self.processed_image = None
        self.face_detected = False

        # Default enhancement parameters
        self.brightness = 1.0
        self.sharpness = 1.0
        self.contrast = 1.0

        # Create GUI elements
        self.create_gui()

    def create_gui(self):
        # Frame for image display and status
        self.status_label = tk.Label(self.root, text="Load an image to start.", font=("Arial", 14))
        self.status_label.grid(row=0, column=0, columnspan=3, pady=10)

        self.image_frame = tk.Label(self.root)
        self.image_frame.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

        # Buttons to load, save images, and copy settings
        tk.Button(self.root, text="Load Image", command=self.load_image).grid(row=2, column=0, padx=5, pady=5)
        tk.Button(self.root, text="Copy Settings", command=self.copy_settings).grid(row=2, column=1, padx=5, pady=5)
        tk.Button(self.root, text="Save Processed Image", command=self.save_image).grid(row=2, column=2, padx=5, pady=5)

        # Sliders for adjustments
        tk.Label(self.root, text="Brightness").grid(row=3, column=0)
        self.brightness_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_brightness
        )
        self.brightness_slider.set(1.0)
        self.brightness_slider.grid(row=3, column=1, columnspan=2)

        tk.Label(self.root, text="Sharpness").grid(row=4, column=0)
        self.sharpness_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_sharpness
        )
        self.sharpness_slider.set(1.0)
        self.sharpness_slider.grid(row=4, column=1, columnspan=2)

        tk.Label(self.root, text="Contrast").grid(row=5, column=0)
        self.contrast_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_contrast
        )
        self.contrast_slider.set(1.0)
        self.contrast_slider.grid(row=5, column=1, columnspan=2)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.png;*.jpeg")])
        if not file_path:
            return
        self.image_path = file_path
        self.original_image = cv2.cvtColor(cv2.imread(self.image_path), cv2.COLOR_BGR2RGB)
        self.processed_image = self.original_image.copy()
        self.apply_enhancements()

    def display_image(self, image, face_locations=None):
        img = image.copy()
        if face_locations:
            for top, right, bottom, left in face_locations:
                cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(img)
        self.image_frame.imgtk = imgtk
        self.image_frame.configure(image=imgtk)

    def apply_enhancements(self):
        if self.original_image is None:
            return
        pil_image = Image.fromarray(self.original_image)
        pil_image = ImageEnhance.Brightness(pil_image).enhance(self.brightness)
        pil_image = ImageEnhance.Sharpness(pil_image).enhance(self.sharpness)
        pil_image = ImageEnhance.Contrast(pil_image).enhance(self.contrast)
        self.processed_image = np.array(pil_image)

        # Detect faces automatically after applying enhancements
        face_locations = face_recognition.face_locations(self.processed_image)
        self.face_detected = len(face_locations) > 0

        # Update status label
        if self.face_detected:
            self.status_label.config(text="Face(s) Detected!", fg="green")
        else:
            self.status_label.config(text="No Face Detected", fg="red")

        self.display_image(self.processed_image, face_locations)

    def update_brightness(self, value):
        self.brightness = float(value)
        self.apply_enhancements()

    def update_sharpness(self, value):
        self.sharpness = float(value)
        self.apply_enhancements()

    def update_contrast(self, value):
        self.contrast = float(value)
        self.apply_enhancements()

    def copy_settings(self):
        settings = f"Brightness: {self.brightness}, Sharpness: {self.sharpness}, Contrast: {self.contrast}"
        self.root.clipboard_clear()
        self.root.clipboard_append(settings)
        self.root.update()  # Keep the clipboard content
        messagebox.showinfo("Settings Copied", "Settings copied to clipboard!")

    def save_image(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to save.")
            return
        save_path = "processed_image.jpg"
        cv2.imwrite(save_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))
        messagebox.showinfo("Image Saved", f"Image saved to {save_path}")


if __name__ == "__main__":
    root = tk.Tk()
    app = FaceDetectionTool(root)
    root.mainloop()
